In [0]:
#Import the Gods 
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms

In [0]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #28x28x1 |3x3x16 ? 28x28x16 ? 3
        self.bnm1 = nn.BatchNorm2d(16)
        self.conv2 = nn.Conv2d(16, 32, 3, padding=1) #28x28x16 | 3x3x32 ? 28x28x32 ? 5
        self.bnm2 = nn.BatchNorm2d(32)
        self.conv3 = nn.Conv2d(32, 16, 1) #28X28X32 | 1x1x16 ? 28x28x16 ?5
        self.bnm3 = nn.BatchNorm2d(16)
        self.pool1 = nn.MaxPool2d(2, 2) #28X28X16 ? 14x14x16 ?10
        self.do1 =  nn.Dropout(p=0.05)

        self.conv4 = nn.Conv2d(16, 16, 3, padding=1) #14x14x16 | 3x3x16 ? 14x14x16 ? 12
        self.bnm5 = nn.BatchNorm2d(16)
        self.do2 =  nn.Dropout(p=0.05)
        self.conv5 = nn.Conv2d(16, 16, 3, padding=1) #14x14x16 | 3x3x16 ? 14x14x16 ? 14
        #self.bnm4 = nn.BatchNorm2d(16)
        self.pool2 = nn.MaxPool2d(2, 2) #14x14x16 ? 7x7x16 ? 28
        self.do3 =  nn.Dropout(p=0.05)
        
        self.conv6 = nn.Conv2d(16, 16, 3) #7x7x16 | 3x3x16 ? 5x5x16 ? 30
        self.conv7 = nn.Conv2d(16, 32, 3) #5x5x16 | 3x3x32? 3x3x32 ? 32
        self.conv8 = nn.Conv2d(32, 10, 3) #3x3x32 | 3x3x10 ? 1x1x10 ? 34

    def forward(self, x):
        x = self.bnm3(F.relu(self.conv3(self.bnm2(F.relu(self.conv2(self.bnm1(F.relu(self.conv1(x)))))))))
        x = self.do1(self.pool1(x))
        x = F.relu(self.conv5(self.do2(self.bnm5(F.relu(self.conv4(x))))))
        x = self.pool2(x)#self.bnm4(x))
        x = self.do3(x)
        x = F.relu(self.conv8(F.relu(self.conv7(F.relu(self.conv6(x))))))#
        #x =  nn.AvgPool2d(x, x.size()[2:])
        x = x.view(-1, 10)
        return F.log_softmax(x)

In [9]:
!pip install torchsummary
from torchsummary import summary
use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 16, 28, 28]             160
       BatchNorm2d-2           [-1, 16, 28, 28]              32
            Conv2d-3           [-1, 32, 28, 28]           4,640
       BatchNorm2d-4           [-1, 32, 28, 28]              64
            Conv2d-5           [-1, 16, 28, 28]             528
       BatchNorm2d-6           [-1, 16, 28, 28]              32
         MaxPool2d-7           [-1, 16, 14, 14]               0
           Dropout-8           [-1, 16, 14, 14]               0
            Conv2d-9           [-1, 16, 14, 14]           2,320
      BatchNorm2d-10           [-1, 16, 14, 14]              32
          Dropout-11           [-1, 16, 14, 14]               0
           Conv2d-12           [-1, 16, 14, 14]           2,320
        MaxPool2d-13             [-1, 16, 7, 7]               0
          Dropout-14             [-1, 1

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:34: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.


In [0]:
torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [0]:
from tqdm import tqdm
def train(model, device, train_loader, optimizer, epoch):
    model.train()
    pbar = tqdm(train_loader)
    for batch_idx, (data, target) in enumerate(pbar):
        data, target = data.to(device), target.to(device)
        optimizer.zero_grad()
        output = model(data)
        loss = F.nll_loss(output, target)
        loss.backward()
        optimizer.step()
        pbar.set_description(desc= f'loss={loss.item()} batch_id={batch_idx}')


def test(model, device, test_loader):
    model.eval()
    test_loss = 0
    correct = 0
    with torch.no_grad():
        for data, target in test_loader:
            data, target = data.to(device), target.to(device)
            output = model(data)
            test_loss += F.nll_loss(output, target, reduction='sum').item()  # sum up batch loss
            pred = output.argmax(dim=1, keepdim=True)  # get the index of the max log-probability
            correct += pred.eq(target.view_as(pred)).sum().item()

    test_loss /= len(test_loader.dataset)

    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.1f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        100. * correct / len(test_loader.dataset)))

In [13]:
model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)

for epoch in range(1, 19):
    train(model, device, train_loader, optimizer, epoch)
    test(model, device, test_loader)

  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.3030, Accuracy: 8791/10000 (87.9%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0397, Accuracy: 9872/10000 (98.7%)



  0%|          | 0/469 [00:00<?, ?it/s]

...


Test set: Average loss: 0.0203, Accuracy: 9939/10000 (99.4%)



  0%|          | 0/469 [00:00<?, ?it/s]


Test set: Average loss: 0.0260, Accuracy: 9923/10000 (99.2%)



loss=0.05974816903471947 batch_id=468: 100%|██████████| 469/469 [00:10<00:00, 43.51it/s]



Test set: Average loss: 0.0201, Accuracy: 9943/10000 (99.4%)

